# testes chroma

- [Chroma DB Tutorial: A Step-By-Step Guide](https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide)

In [ ]:
import chromadb
from chromadb.config import Settings


#client = chromadb.Client(Settings(persist_directory="chroma/testes/"))
client = chromadb.PersistentClient(path="../chroma/testes/", settings=Settings(allow_reset=True))
collection = client.create_collection(name="Students")

In [ ]:
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [ ]:
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
results = collection.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

# Embeddings

In [ ]:
from langchain_ollama import OllamaEmbeddings 
from chromadb.utils import embedding_functions

embedding_model = OllamaEmbeddings(model="nomic-embed-text")
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#                 model_name="text-embedding-ada-002"
#             )
#students_embeddings = openai_ef([student_info, club_info, university_info])
students_embeddings = embedding_model.embed_documents([student_info, club_info, university_info])
print(students_embeddings)

# obs: precisa do $ ollama serve

In [ ]:
collection2 = client.get_or_create_collection(name="Students2")

collection2.add(
    embeddings = students_embeddings,
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
class CustomOllamaEmbeddingFunction:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model

    def __call__(self, input): # 'input' é o nome esperado pelo ChromaDB
        return self.embedding_model.embed_documents(input) # Passa 'input' para embed_documents


custom_embedding_function = CustomOllamaEmbeddingFunction(embedding_model)
collection2 = client.get_or_create_collection(name="Students2",embedding_function=custom_embedding_function )
# collection2 = client.get_or_create_collection(name="Students2",embedding_function=openai_ef)

collection2.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
results = collection2.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

# Updating and Removing Data

In [ ]:
collection2.update(
    ids=["id1"],
    documents=["Kristiane Carina, a 19-year-old computer science sophomore with a 3.7 GPA"],
    metadatas=[{"source": "student info"}],
)

In [ ]:
results = collection2.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

In [ ]:
collection2.delete(ids = ['id1'])


results = collection2.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

# Collection Management

In [ ]:
vector_collections = client.create_collection("vectordb")


vector_collections.add(
    documents=["This is Chroma DB CheatSheet",
               "This is Chroma DB Documentation",
               "This document Chroma JS API Docs"],
    metadatas=[{"source": "Chroma Cheatsheet"},
    {"source": "Chroma Doc"},
    {'source':'JS API Doc'}],
    ids=["id1", "id2", "id3"]
)

In [ ]:
vector_collections.count()

In [ ]:
vector_collections.get()

In [ ]:
vector_collections.modify(name="chroma_info")

# list all collections
client.list_collections()

In [ ]:
vector_collections_new = client.get_collection(name="chroma_info")

In [ ]:
client.delete_collection(name="chroma_info")
client.list_collections()

In [ ]:
client.reset()
client.list_collections()